In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from sklearn.model_selection import train_test_split


In [1]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 MB 14.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 15.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 16.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 15.8 MB/s eta 0:00:00a 0:00:01


In [5]:
# Load ratings dataset
ratings = pd.read_csv("ml-latest-small/ratings.csv")

# Scale ratings between 0 and 1 (implicit feedback)
ratings['rating'] = ratings['rating'].apply(lambda x: 1 if x >= 3.5 else 0)

# Get unique user and movie IDs
user_ids = ratings['userId'].unique().tolist()
movie_ids = ratings['movieId'].unique().tolist()

# Map users and movies to indices
user_id_map = {id: idx for idx, id in enumerate(user_ids)}
movie_id_map = {id: idx for idx, id in enumerate(movie_ids)}

# Replace original IDs with mapped indices
ratings['userId'] = ratings['userId'].map(user_id_map)
ratings['movieId'] = ratings['movieId'].map(movie_id_map)

# Split into training and testing sets
train, test = train_test_split(ratings, test_size=0.2, random_state=42)


In [7]:
# Define model parameters
num_users = len(user_id_map)
num_movies = len(movie_id_map)
embedding_dim = 50  # Dimensionality of embedding space

# Define input layers
user_input = Input(shape=(1,), name='User_Input')
movie_input = Input(shape=(1,), name='Movie_Input')

# Create embedding layers
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name='User_Embedding')(user_input)
movie_embedding = Embedding(input_dim=num_movies, output_dim=embedding_dim, name='Movie_Embedding')(movie_input)

# Flatten embeddings
user_vector = Flatten()(user_embedding)
movie_vector = Flatten()(movie_embedding)

# Concatenate user and movie embeddings
concat = Concatenate()([user_vector, movie_vector])

# Fully connected layers
dense_1 = Dense(128, activation='relu')(concat)
dropout_1 = Dropout(0.3)(dense_1)
dense_2 = Dense(64, activation='relu')(dropout_1)
dropout_2 = Dropout(0.3)(dense_2)
dense_3 = Dense(32, activation='relu')(dropout_2)

# Output layer (binary classification: liked/not liked)
output = Dense(1, activation='sigmoid')(dense_3)

# Build and compile model
ncf_model = Model(inputs=[user_input, movie_input], outputs=output)
ncf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Show model summary
ncf_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ User_Input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Movie_Input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ User_Embedding      │ (None, 1, 50)     │     30,500 │ User_Input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Movie_Embedding     │ (None, 1, 50)     │    486,200 │ Movie_Input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 50)        │          0 │ User_Embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 50)        │          0 │ Movie_Embedding[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 100)       │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     12,928 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      2,080 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         33 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 539,997 (2.06 MB)

 Trainable params: 539,997 (2.06 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Prepare training data
train_user_input = train['userId'].values
train_movie_input = train['movieId'].values
train_labels = train['rating'].values  # 1 for liked, 0 for not liked

# Train model
history = ncf_model.fit(
    [train_user_input, train_movie_input], train_labels,
    epochs=10, batch_size=64, validation_split=0.1, verbose=1
)


Epoch 1/10
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6785 - loss: 0.5950 - val_accuracy: 0.7259 - val_loss: 0.5413
Epoch 2/10
 349/1135 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7723 - loss: 0.4746

In [ ]:
# Prepare test data
test_user_input = test['userId'].values
test_movie_input = test['movieId'].values
test_labels = test['rating'].values

# Get predictions
predictions = ncf_model.predict([test_user_input, test_movie_input])
predicted_ratings = (predictions > 0.5).astype(int)

# Compute Accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predicted_ratings)
print(f"NCF Accuracy: {accuracy:.4f}")


In [6]:
from collections import defaultdict

# Function to compute Precision@K and Recall@K
def precision_recall_at_k(y_true, y_pred, k=10):
    user_ratings = defaultdict(list)

    for uid, actual, pred in zip(test['userId'], y_true, y_pred):
        user_ratings[uid].append((actual, pred))

    precisions, recalls = [], []

    for uid, ratings in user_ratings.items():
        ratings.sort(key=lambda x: x[1], reverse=True)  # Sort by predicted score
        top_k = ratings[:k]

        relevant = sum((actual == 1) for actual, _ in ratings)
        recommended_relevant = sum((actual == 1) for actual, _ in top_k)

        precision = recommended_relevant / k
        recall = recommended_relevant / relevant if relevant > 0 else 0

        precisions.append(precision)
        recalls.append(recall)

    return np.mean(precisions), np.mean(recalls)

# Compute Precision@K and Recall@K for NCF
precision_ncf, recall_ncf = precision_recall_at_k(test_labels, predictions, k=10)
print(f"NCF Precision@10: {precision_ncf:.4f}, Recall@10: {recall_ncf:.4f}")


NCF Precision@10: 0.6005, Recall@10: 0.6496


In [10]:
import numpy as np


def ndcg_at_k(y_true, y_pred, k):
    ndcg_values = []

    # Loop through each user and their labels/predictions
    for actual, pred in zip(y_true, y_pred):
        # Ensure actual is a list or array
        if not isinstance(actual, (list, np.ndarray)):
            actual = [actual]  # Wrap single values in a list

        # Sort both by relevance
        ideal_ranking = sorted(actual, reverse=True)[:k]
        pred_ranking = [x for _, x in sorted(zip(pred, actual), reverse=True)][:k]

        # Compute DCG
        dcg = sum((rel / np.log2(i + 2)) for i, rel in enumerate(pred_ranking))
        # Compute IDCG
        idcg = sum((rel / np.log2(i + 2)) for i, rel in enumerate(ideal_ranking))
        # Handle division by zero in case idcg is 0
        ndcg = dcg / idcg if idcg > 0 else 0
        ndcg_values.append(ndcg)

    return np.mean(ndcg_values)


# Dummy data for testing
test_labels = [[3, 2, 3, 0, 1], [2, 1, 0, 0, 0]]  # True relevance scores per user
predictions = [[0.9, 0.8, 0.6, 0.5, 0.4], [0.7, 0.6, 0.2, 0.1, 0.0]]  # Predicted scores per user

# Compute NDCG at k=10
ndcg_ncf = ndcg_at_k(test_labels, predictions, k=10)
print(f"NCF NDCG@10: {ndcg_ncf:.4f}")


NCF NDCG@10: 0.9862
